In [1]:
using LinearAlgebra
using Kronecker

In [38]:
######################
#### variables.jl ####
######################

γ = -1 # Gyromagnetic ratio for an electron (T^-1s^-1)
Bext = 10 # External magnetic field (T)
ωL = abs(γ)*Bext # Larmor frequency (s^-1)
Λ = 10^10 # Cutoff frequency for spectral density

### Temperatures ###

### Statics ###
T = exp10.(range(-3, 3, length=100));

### Dynamics ###

TDyn = 1

### Parameter Sets ###
prma = [1.4 0.001 10]
prmb = [7 0 50]
prmc = [0.014 0 0.1]
prmd = [1.4 0 10000]
prme = [2 0.001 10]
prmf = [2 0.001 1]
prmg = [2 0.001 50]

prm1 = prme; # Change the RHS here to change parameter set for coupling direction 1
prm2 = prme; # Change the RHS here to change parameter set for coupling direction 2
prm3 = prme; # Change the RHS here to change parameter set for coupling direction 3

ω01, Γ1, α1 = [2.0 0.001 10.0]
ω02, Γ2, α2 = prm2
ω03, Γ3, α3 = prm3

### RC-Specific Parameters ###

Ω1, Ω2, Ω3 = [ω01 ω02 ω03]
λ1, λ2, λ3 = [sqrt(α1/Ω1) sqrt(α2/Ω2) sqrt(α3/Ω3)]
δ1, δ2, δ3 = [Γ1 Γ2 Γ3]
n1, n2, n3 = [3 3 1] # Change the number of RC levels here

δ_list(dim) = [2.0 δ2 δ3][dim] # List of dissipation strengths for Dynamics.jl module
hspace_dimension(dim) = [2*n1 2*n1*n2 2*n1*n2*n3][dim] # List of Hilbert space dimensions

hspace_dimension (generic function with 1 method)

In [3]:
###########################
#### initial_States.jl ####
###########################

### Spins ###

## Pauli Matrices ##
σx = [[0 1];[1 0]]
σy = [[0 -im];[im 0]]
σz = [[1 0];[0 -1]]

scale = 2;

## Spin Coupling Operators ##
# The {θ, ϕ}[i] pairs set the direction of coupling in the ith direction #
function sc(i)
    θ = [0 0 0] # Coupling direction 1, 2 and 3
    ϕ = [0 0 0] # Coupling direction 1, 2 and 3 
    # # 1D x - Coupling ##
    # θ = [π/2 0 0]
    # ϕ = [0 0 0]
    # # 1D y - Coupling ##
    # θ = [π/2 0 0]
    # ϕ = [π/2 0 0] 
    # # 1D z- Coupling ##
    # θ = [0 0 0]
    # ϕ = [0 0 0]
    return σx*(sin(θ[i])*cos(ϕ[i])) + σy*(sin(θ[i])*sin(ϕ[i])) + σz*cos(θ[i])
end

## Bloch State ##
# This sets the initial state of the spin on the Bloch sphere, defined by the polar angle α and azimuth β #
function bloch_state()
    α = -π/2
    β = 0
    return [cos(α/2)^2 0.5*exp(-im*β)*sin(α); 0.5*exp(im*β)*sin(α) sin(α/2)^2]
end

### RC ###

## Thermal Initial State ##
function gibbs(H, T)
    n = size(H, 1)
    ϵ = eigen(H).values
    P = eigen(H).vectors
    𝒵 = sum(exp(-ϵ[i]/T) for i = 1:n)
    ρ = (1/𝒵)*Diagonal([exp(-ϵ[i]/T) for i = 1:n])
    return P*ρ*adjoint(P)
end

### Joint Initital States ###
function ρ0(dim)
    if dim == 1
        return kronecker(bloch_state(), gibbs(HB(n1, Ω1), TDyn))
    elseif dim == 2
        return kronecker(bloch_state(), gibbs(HB(n1, Ω1), TDyn), gibbs(HB(n2, Ω2), TDyn))
    elseif dim == 3
        return kronecker(bloch_state(), gibbs(HB(n1, Ω1), TDyn), gibbs(HB(n2, Ω2), TDyn), gibbs(HB(n3, Ω3), TDyn))
    else
        print("Please return a dimension of either 1, 2 or 3.")
    end
end

ρ0 (generic function with 1 method)

In [4]:
############################
#### spectralDensity.jl ####
############################

### Spectral Density ###
spectral_density(ω, δ) = ((δ*ω)/π)*(Λ^2/(ω^2 + Λ^2))

spectral_density (generic function with 1 method)

In [5]:
##################
#### maths.jl ####
##################

### Commutators/Anticommutators ###
comm(A,B) = A*B - B*A
acomm(A,B) = A*B + B*A

### Square ###
square(n) = n*n

### Identity Matrices ###
𝕀(n) = Matrix(I, n, n) 
𝕀s = 𝕀(2) # Spin

#### Partial Trace ###
function ptrace(ρ, n)
    nR = Int(size(ρ, 1)/n) # This is the remaining dimension
    lhs(i) = kronecker(𝕀(nR), (𝕀(n)[[i],:]))
    rhs(i) = kronecker(𝕀(nR), (𝕀(n)[:,i]))
    return sum(lhs(i)*ρ*rhs(i) for i=1:n)
end

### Uhlmann Fidelity ###
fidelity(ρ1, ρ2) = square(tr(sqrt(sqrt(ρ1)*ρ2*sqrt(ρ1))))

### Check for Choppable Components ###
realIfClose(c) = isnan(imag(c)) || imag(c) < 1e-14 ? real(c) : c;
realIfClose(c::AbstractArray) = realIfClose.(c);

In [6]:
#########################
#### hamiltonians.jl ####
#########################

### Creation and Annihilation Operators ###

## Creation Operator ##
function create(n)
    matrix = zeros(n, n)
    for i in 1:n
        new_row = zeros(1, n)
        for j in Array(1:n)
            if i == j+1
                new_row[j] = sqrt(i-1)
            else
                new_row[j] = 0
            end
        end
        matrix[[i],:] = new_row
    end
    return matrix
end

## Annihilation Operator ##
function annihilate(n)
    return adjoint(create(n))
end

### Hamiltonians ###

## Bare Spin Hamiltonian ##
HG() = -sign(γ)*σz

## 1D Hamiltonian ##
function HS1D(n1, λ1, Ω1)
    spin = -sign(γ)*kronecker(σz, 𝕀(n1))
    rc = scale*kronecker(𝕀s, Ω1*(create(n1)*annihilate(n1)))
    int = λ1*kronecker(sc(1), (create(n1) + annihilate(n1)))
    return spin + rc + int
end

## 2D Hamiltonian ##
function HS2D(n1, n2, λ1, λ2, Ω1, Ω2)
    spin = -sign(γ)*kronecker(σz, 𝕀(n1), 𝕀(n2))
    rc = scale*kronecker(𝕀s, (Ω1/ωL)*(create(n1)*annihilate(n1)), 𝕀(n2)) + scale*kronecker(𝕀s, 𝕀(n1), (Ω2/ωL)*(create(n2)*annihilate(n2)))
    int = (λ1/ωL)*kronecker(sc(1), (create(n1) + annihilate(n1)), 𝕀(n2)) + (λ2/ωL)*kronecker(sc(2), 𝕀(n1), (create(n2) + annihilate(n2)))
    return spin + rc + int
end

## 3D Hamiltonian ##
function HS3D(n1, n2, n3, λ1, λ2, λ3, Ω1, Ω2, Ω3)
    spin = -sign(γ)*kronecker(σz, 𝕀(n1), 𝕀(n2), 𝕀(n3))
    rc = scale*kronecker(𝕀s, (Ω1/ωL)*(create(n1)*annihilate(n1)), 𝕀(n2), 𝕀(n3)) + scale*kronecker(𝕀s, 𝕀(n1), (Ω2/ωL)*(create(n2)*annihilate(n2)), 𝕀(n3)) + scale*kronecker(𝕀s, 𝕀(n1), 𝕀(n2), (Ω3/ωL)*(create(n3)*annihilate(n3)))
    int = λ1*kronecker(sc(1), (create(n1) + annihilate(n1)), 𝕀(n2), 𝕀(n3)) + λ2*kronecker(sc(2), 𝕀(n1), (create(n2) + annihilate(n2)), 𝕀(n3)) + λ3*kronecker(sc(3), 𝕀(n1), 𝕀(n2), (create(n3) + annihilate(n3)))
    return spin + rc + int
end

## Single Function for Access to All Dimensions ##
function HS(dim)
    if dim == 1
        return HS1D(n1, λ1, Ω1)
    elseif dim == 2
        return HS2D(n1, n2, λ1, λ2, Ω1, Ω2)
    elseif dim == 3 
        return HS3D(n1, n2, n3, λ1, λ2, λ3, Ω1, Ω2, Ω3)
    else
        print("Please return a dimension of either 1, 2 or 3.")
    end
end

## General Bath Hamiltonian ##
HB(n, Ω) = Ω*(create(n)*annihilate(n))

HB (generic function with 1 method)

In [12]:
X(n) = create(n) + annihilate(n)

# 1D Coupling #
jump1D(n1) = [kronecker(𝕀s, X(n1))]

# 2D Coupling #
jump2D(n1, n2) = [kronecker(𝕀s, X(n1), 𝕀(n2)), kronecker(𝕀s, 𝕀(n1), X(n2))]

# 3D Coupling #
jump3D(n1, n2, n3) = [kronecker(𝕀s, X(n1), 𝕀(n2), 𝕀(n3)), kronecker(𝕀s, 𝕀(n1), X(n2), 𝕀(n3)), kronecker(𝕀s, 𝕀(n1), 𝕀(n2), X(n3))] 

jump3D (generic function with 1 method)

In [13]:
function jump(dim)
    if dim == 1
        return jump1D(n1)
    elseif dim == 2
        return jump2D(n1, n2)
    elseif dim == 3
        return jump3D(n1, n2, n3)
    else
        print("Please return a dimension of either 1, 2 or 3.")
    end
end

jump (generic function with 1 method)

In [14]:
HS1D(2, sqrt(10.0/2.0), 2.0)

4×4 Matrix{ComplexF64}:
     1.0+0.0im  2.23607+0.0im       0.0-0.0im       0.0-0.0im
 2.23607+0.0im      5.0+0.0im       0.0-0.0im       0.0-0.0im
     0.0+0.0im      0.0+0.0im      -1.0+0.0im  -2.23607+0.0im
     0.0+0.0im      0.0+0.0im  -2.23607+0.0im       3.0+0.0im

In [30]:
function transitions(dim, i) # dim is the coupling dimension (1 for 1D, 2 for 2D etc...) and i the specific RC to generate Bohr frequencies/jump operators for
    H = HS1D(2, sqrt(10.0/2.0), 2.0)
    A = jump1D(2)[1]
    n = 4
    table = zeros(n, n)
    bohr_freqs = Float64[]
    jump_ops = Any[]
    eval = eigen(H).values
    evec = eigen(H).vectors
    proj(j) = evec[:,j]*adjoint(evec[:,j])
    # Create a table of transition frequencies
    for k in 1:n
        for l in 1:n
            table[k, l] = eval[l] - eval[k]
            if table[k, l] !=  0.0
                append!(bohr_freqs, table[k, l])
                push!(jump_ops, proj(l)*A*proj(k))
            end
        end
    end
    return bohr_freqs, jump_ops
end

transitions (generic function with 1 method)

In [39]:
function 𝒮(dim)

    ## Useful Definitions ##
    H = HS1D(2, sqrt(10.0/2.0), 2.0) # System Hamiltonian for given dimension of coupling
    n = hspace_dimension(dim) # Dimension of Hilbert space for given coupling dimension

    ## Bohr Frequencies/Jump Operators for Each Coupling Dimension ##
    transitions_list(i) = transitions(dim, i) # Extract Bohr freq-jump op pairs for the ith bath
    len(i) = length(transitions_list(i)[1]) # Find the length of list (for each bath) to iterate over
    ωb(i) = transitions_list(i)[1] # Rewrite function outputs (Bohr freqs) in more compact form
    ATr(i) = transitions_list(i)[2] # Rewrite function outputs (transformed jump ops) in more compact form
    ATot(i) = sum(ATr(i)[j] for j = 1:len(i)) # Define the sum of all jump operators for each bath

    ## Iles-Smith Superoperators ##
    χ(i) = (π/2)*sum(spectral_density(ωb(i)[j], δ_list(i))*coth((ωb(i)[j])/(2*TDyn))*ATr(i)[j] for j = 1:len(i)) # Iles-Smith χ superoperator
    Θ(i) = (π/2)*sum(spectral_density(ωb(i)[j], δ_list(i))*ATr(i)[j] for j = 1:len(i)) # Iles-Smith Θ superoperator

    ## Left/Right Multiplication Superoperators ##
    ℒ(operator) = kronecker(operator, 𝕀(n)) # Define the left multiplication superoperator
    ℛ(operator) = kronecker(𝕀(n), transpose(operator)) # Define the right multiplication superoperator

    ## Return the Superoperator ##
    return -im*(ℒ(H) - ℛ(H)) + sum(- ℒ(ATot(i))*(ℒ(χ(i)) - ℛ(χ(i))) + ℛ(ATot(i))*(ℒ(χ(i)) - ℛ(χ(i))) + ℒ(ATot(i))*(ℒ(Θ(i)) + ℛ(Θ(i))) - ℛ(ATot(i))*(ℒ(Θ(i)) + ℛ(Θ(i))) for i in 1:dim)

end

𝒮 (generic function with 1 method)

In [43]:
H = HS1D(2, sqrt(10.0/2.0), 2.0) # System Hamiltonian for given dimension of coupling
n = hspace_dimension(1) # Dimension of Hilbert space for given coupling dimension

## Bohr Frequencies/Jump Operators for Each Coupling Dimension ##
transitions_list(i) = transitions(1, i) # Extract Bohr freq-jump op pairs for the ith bath
len(i) = length(transitions_list(i)[1]) # Find the length of list (for each bath) to iterate over
ωb(i) = transitions_list(i)[1] # Rewrite function outputs (Bohr freqs) in more compact form
ATr(i) = transitions_list(i)[2] # Rewrite function outputs (transformed jump ops) in more compact form
ATot(i) = sum(ATr(i)[j] for j = 1:len(i)) # Define the sum of all jump operators for each bath

ATot (generic function with 1 method)

In [44]:
χ(i) = (π/2)*sum(spectral_density(ωb(i)[j], δ_list(i))*coth((ωb(i)[j])/(2*TDyn))*ATr(i)[j] for j = 1:len(i)) # Iles-Smith χ superoperator

χ (generic function with 1 method)

In [45]:
χ(1)

4×4 Matrix{ComplexF64}:
 2.99624+0.0im   2.67992+0.0im       0.0+0.0im      0.0+0.0im
 2.67992+0.0im  -2.99624+0.0im       0.0+0.0im      0.0+0.0im
     0.0+0.0im       0.0+0.0im  -2.99624+0.0im  2.67992+0.0im
     0.0+0.0im       0.0+0.0im   2.67992+0.0im  2.99624+0.0im